# Surprise homework

In [1]:
import io 
from collections import defaultdict

import pandas as pd
from surprise import Dataset
from surprise import SVD
from surprise import NormalPredictor
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import cross_validate

from surprise.model_selection import train_test_split

### Functions

In [2]:
# используйте полезные функции из FAQ
def get_top_n(predictions, n=5):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def precision_recall_at_k(predictions, k=5, threshold=3.52):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return pd.Series(precisions).mean(), pd.Series(recalls).mean()

### Load data

In [3]:
data = Dataset.load_builtin('ml-100k')
data

### Define algorithms

In [4]:
# внимательно изучите документацию по метрикам и алгоритмам
algoNorm = NormalPredictor()

sim_options = {'name': 'cosine', 'user_based': True}
algoCos = KNNWithMeans(sim_options=sim_options)

sim_options = {'name': 'msd', 'user_based': True}
algoMSD = KNNWithMeans(im_options=sim_options)

sim_options = {'name': 'pearson', 'user_based': True}
algoPearson = KNNWithMeans(sim_options=sim_options)

algoSVD = SVD()

### Select algorithm

In [5]:
cross_validate(algoNorm, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5199  1.5287  1.5167  1.5233  1.5205  1.5218  0.0040  
Fit time          0.21    0.22    0.23    0.22    0.25    0.23    0.01    
Test time         0.25    0.29    0.24    0.30    0.27    0.27    0.02    


{'test_rmse': array([1.51986826, 1.52872023, 1.51671042, 1.52334935, 1.52046452]),
 'fit_time': (0.21299505233764648,
  0.22063922882080078,
  0.22925543785095215,
  0.22224116325378418,
  0.2521946430206299),
 'test_time': (0.25333428382873535,
  0.29030442237854004,
  0.23680830001831055,
  0.30280303955078125,
  0.26595211029052734)}

In [6]:
cross_validate(algoCos, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9615  0.9575  0.9498  0.9496  0.9564  0.9550  0.0046  
Fit time          2.67    2.79    3.66    3.37    3.00    3.10    0.37    
Test time         6.34    6.45    8.14    9.08    9.29    7.86    1.26    


{'test_rmse': array([0.96153086, 0.95745649, 0.94981036, 0.9496224 , 0.95643475]),
 'fit_time': (2.6702208518981934,
  2.792437791824341,
  3.6598093509674072,
  3.372833728790283,
  3.000225067138672),
 'test_time': (6.338767766952515,
  6.451610088348389,
  8.144773483276367,
  9.081602573394775,
  9.288647890090942)}

In [7]:
cross_validate(algoMSD, data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9512  0.9444  0.9538  0.9528  0.9526  0.9510  0.0034  
Fit time          1.70    0.89    0.92    1.16    0.92    1.12    0.31    
Test time         9.27    6.69    7.00    6.67    6.88    7.30    0.99    


{'test_rmse': array([0.95117656, 0.94440351, 0.95377776, 0.95277482, 0.95263688]),
 'fit_time': (1.7018496990203857,
  0.8933818340301514,
  0.9242129325866699,
  1.1573352813720703,
  0.9240150451660156),
 'test_time': (9.273337364196777,
  6.6918041706085205,
  6.998839378356934,
  6.6679792404174805,
  6.8802690505981445)}

In [8]:
cross_validate(algoPearson, data, measures=['RMSE'], cv=5, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9513  0.9502  0.9535  0.9448  0.9433  0.9486  0.0039  
Fit time          3.50    3.38    3.30    3.37    3.31    3.37    0.07    
Test time         7.00    6.35    6.67    7.11    6.57    6.74    0.28    


{'test_rmse': array([0.95126424, 0.95016787, 0.95350142, 0.94478219, 0.94333238]),
 'fit_time': (3.503183126449585,
  3.38485050201416,
  3.29801344871521,
  3.371081590652466,
  3.3067855834960938),
 'test_time': (6.999757528305054,
  6.350005865097046,
  6.673359394073486,
  7.114450454711914,
  6.5724005699157715)}

In [9]:
cross_validate(algoSVD, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9331  0.9354  0.9467  0.9388  0.9251  0.9358  0.0071  
Fit time          8.86    8.60    8.77    8.31    8.39    8.59    0.21    
Test time         0.22    0.31    0.31    0.24    0.33    0.28    0.05    


{'test_rmse': array([0.93305745, 0.93543967, 0.94671381, 0.93876967, 0.925051  ]),
 'fit_time': (8.863641262054443,
  8.595730304718018,
  8.770361423492432,
  8.311644077301025,
  8.392230749130249),
 'test_time': (0.2173633575439453,
  0.3116447925567627,
  0.3147110939025879,
  0.2399890422821045,
  0.32941579818725586)}

### Calculate precision@k and recall@k

In [10]:
trainset, testset = train_test_split(data, test_size=.25)
algoSVD.fit(trainset)
predictions = algoSVD.test(testset)

print(precision_recall_at_k(predictions))

(0.7283315659243549, 0.35982176379396313)


### Predict

In [11]:
# обратите внимание на функцию build_anti_testset
user = '1'
user_predict = get_top_n(predictions)[user]

In [12]:
item_path = '/Users/Admin/.surprise_data/ml-100k/ml-100k/u.item'
item_df = pd.read_csv(item_path, sep='|',encoding='ISO-8859-1', header = None, index_col = 0)

In [13]:
print("User " + user)
for item in user_predict:
    film = item_df.loc[int(item[0])][1]
    date = item_df.loc[int(item[0])][2]
    print(item[0] + " ('" + film + "', '" + date + "') " + str(round(item[1], 3)))

User 1
50 ('Star Wars (1977)', '01-Jan-1977') 4.734
114 ('Wallace & Gromit: The Best of Aardman Animation (1996)', '05-Apr-1996') 4.591
151 ('Willy Wonka and the Chocolate Factory (1971)', '01-Jan-1971') 4.59
173 ('Princess Bride, The (1987)', '01-Jan-1987') 4.538
228 ('Star Trek: The Wrath of Khan (1982)', '01-Jan-1982') 4.497
